In [1]:
import numpy as np
import os
import sys
from pysr import PySRRegressor
from sympy import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
COUNTER_FILENAME = os.path.expanduser("../bigan/model_counter.txt")
global MODEL_ID

# Adjust printing view dimensions
np.set_printoptions(threshold=sys.maxsize, linewidth=300)

def retrieve_model_id():
    global MODEL_ID
    try:
        with open(COUNTER_FILENAME, 'r') as f:
            count = f.read()
            MODEL_ID = count
    except FileNotFoundError:
        print('New counter file.')
        with open(COUNTER_FILENAME, 'w') as f:
            count = '0'
            f.write(count)
            MODEL_ID = count

            
retrieve_model_id()

LOAD_PATH =  r'..\bigan\data\activations\ae_{}'.format(MODEL_ID)
params = np.load(r'{}\params.npy'.format(LOAD_PATH))
activs = np.load(r'{}\activs.npy'.format(LOAD_PATH))
losses = np.load(r'{}\losses.npy'.format(LOAD_PATH))

# Remove inactive neurons
activs = np.delete(activs, 1, 1)
activs = np.delete(activs, 2, 1)

X = params
y = activs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=14)

In [3]:
model = PySRRegressor(
    model_selection="best",  # Result is mix of simplicity+accuracy
    niterations=40,
    binary_operators=[
        "+", 
        "-", 
        "*", 
        "/", 
    ],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "log",
    ],
    extra_sympy_mappings={
        "inv": lambda x: 1 / x,
        "sqrt_abs": lambda x: sqrt(Abs(x)),
        "sq_mult": lambda x, y: (x**2) * y,
        "sq": lambda x: x**2,
        "quartic": lambda x: x**4,
    },
    loss="loss(x, y) = (x - y)^2",
)

In [4]:
model.fit(X_train, y_train)

C:\Users\evere\anaconda3\envs\pysr\lib\site-packages\pysr\sr.py:1225: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


PySRRegressor.equations_ = [
[
	   pick      score                         equation          loss  complexity
	0         0.000000                               x3  1.489076e+06           1
	1        14.933817                          sin(x3)  4.866793e-01           2
	2  >>>>   5.073537  ((1.2042814 / x0) / -1.1023118)  1.194032e-07           5
], [
	    pick         score                                           equation  \
	0         0.000000e+00                                          16.606995   
	1         1.129579e-01                                   (x1 * 0.3790391)   
	2         1.042859e-01                  ((x3 * -0.03259458) - -55.977577)   
	3         9.263943e-02              ((0.5710069 - (x2 * 7.4914274)) * x1)   
	4         2.106986e-03           ((0.5710069 - sin(x2 * 7.4914274)) * x1)   
	5         2.257353e-01  ((((x1 / 0.12562343) - x3) * 0.03102481) - -44...   
	6         3.058034e-01  ((((x1 / (x2 + 0.043693945)) - x3) * 0.0227828...   
	7   >>>>  2.094901e-01  ((((x1 / ((x2 * 3.326335) + 0.051082987)) - x3...   
	8         7.211463e-07  ((((x1 / ((x2 * 3.326335) + 0.051082987)) - x3...   
	9         3.226704e-02  (((((x1 - -4.423186) / ((x2 * 3.4527943) + 0.0...   
	10        1.709452e-02  (((((x1 - log_abs(x2)) / ((x2 * 3.1516595) + 0...   
	11        4.765648e-05  (((((x1 - log_abs(x2)) / ((x2 * 3.1516595) + s...   
	12        3.277033e-03  (((((x1 - log_abs(x2 * 0.33252007)) / ((x2 * 3...   
	13        6.219277e-03  ((((((x1 / ((x2 * 3.326335) + 0.051082987)) - ...   
	
	         loss  complexity  
	0   90.712440           1  
	1   72.369050           3  
	2   58.745050           5  
	3   48.809650           7  
	4   48.706917           8  
	5   38.864674           9  
	6   21.083250          11  
	7   13.866816          13  
	8   13.866806          14  
	9   13.426507          15  
	10  13.198938          16  
	11  13.198309          17  
	12  13.155129          18  
	13  13.073567          19  
]]

In [5]:
y_hat = model.predict(X_test)
mean_squared_error(y_test, y_hat)

5.026095587908345

In [10]:
model.latex()

['- \\frac{1.09}{x_{0}}',
 '\\frac{0.0302 x_{1}}{3.33 x_{2} + 0.0511} - 0.0302 x_{3} + 42.5']